In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time,json, dataclasses
from typing import List, Optional
import ulid, re, time
from datetime import datetime as dt, timedelta

In [ ]:
# URL du site
website_path = "https://co.computrabajo.com/"

# Today
today = dt.now()

In [5]:
def create_driver():
    options = Options()
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(2) 
    return driver
driver = create_driver()

In [6]:
computrabajo_creds_path = r"C:\Users\f.gionnane\Documents\Selenium\Jobs announcements\connection_creds.json"
with open(computrabajo_creds_path, 'r') as file:
    content = json.load(file)
    email = content['email']
    password = content['password']

In [7]:
def login(driver, email, password):
    # Cliquer sur le bouton de login
    try:
        driver.get(website_path)
        time.sleep(3)
    
        login_button = driver.find_element(By.XPATH, "//span[@class='b_primary_inv']")
        login_button.click()
        time.sleep(1)
        ingresar_button = driver.find_element(By.XPATH, "//span[@alt='Ingresar']")
        ingresar_button.click()
        time.sleep(1.5)
        email_field = driver.find_element(By.XPATH, "//input[@id='LoginModel_Email']")
        email_field.send_keys(email)
        
        continuar_button = driver.find_element(By.XPATH, "//a[@id='continueWithMailButton']")
        continuar_button.click()
        time.sleep(1)
        contrasena_field = driver.find_element(By.XPATH, "//input[@id='LoginModel_Password']")
        contrasena_field.send_keys(password)
        contrasena_field.send_keys(Keys.ENTER)
        img_head_cv = driver.find_element(By.XPATH, "//img[@name='imgheadcv']")
        if img_head_cv:
            print("Connexion réussie!")
    except Exception as e:
        print(f"Erreur lors de la connexion :\n{e}")

login(driver, email, password)

Connexion réussie!


In [ ]:
today = dt.today()

def search_for_jobs(driver, keywords, location):
    time.sleep(3)  # Laisse le temps à la page de charger

    job_field = driver.find_element(By.XPATH, "//input[@id='prof-cat-search-input']")
    job_field.send_keys(keywords)
    job_field.send_keys(Keys.ENTER)
    time.sleep(1)

    location_bar = driver.find_element(By.XPATH, "//input[@id='place-search-input']")
    location_bar.send_keys(location)
    location_bar.send_keys(Keys.ENTER)
    time.sleep(1)

    try:
        ahora_no_button_xpath = "//button[@onclick='webpush_subscribe_ko(event);']"
        ahora_no_button = driver.find_element(By.XPATH, ahora_no_button_xpath)
        ahora_no_button.click()
        time.sleep(1)
    except:
        pass

def collect_offers(driver):
    offers = []

    try:
        jobs_div = driver.find_elements(By.XPATH, "//article[@class='box_offer']")

        for job in jobs_div:
            offer_dict = {}

            try:
                offer_date_elem = job.find_element(By.CLASS_NAME, "fc_aux")
                offer_date_text = offer_date_elem.text.strip().lower()

                if "ayer" in offer_date_text:
                    offer_date = today - timedelta(days=1)
                elif offer_date_text.startswith("hace") and "horas" in offer_date_text:
                    offer_date = today
                elif offer_date_text.startswith("hace") and "días" in offer_date_text:
                    date_diff = re.search(r'\d+', offer_date_text)
                    offer_date = today - timedelta(days=int(date_diff.group())) if date_diff else today
                else:
                    continue

                offer_dict['offer_date'] = offer_date.strftime('%Y-%m-%d')
            except:
                continue  # skip this job if date is missing or broken

            # Génère une clé unique
            offer_dict['unique_key'] = str(ulid.new())

            try:
                title_elem = job.find_element(By.CLASS_NAME, "title_offer")
                offer_dict['title'] = title_elem.text.strip()
            except:
                offer_dict['title'] = ""

            # Clique pour charger les détails du job
            try:
                job.click()
                time.sleep(1)
            except:
                pass

            try:
                company_elem = driver.find_element(By.CSS_SELECTOR, 'a.dIB.mr10')
                offer_dict['company'] = company_elem.text.strip()
            except:
                offer_dict['company'] = ""
            try:

                location = driver.find_element(By.CSS_SELECTOR, ".fs16.mb5")
                offer_dict['location'] = location

            except:
                offer_dict['location'] = ""

            try:
                contract_section = driver.find_element(By.XPATH, "//div[@class='mbB']")
                parts = contract_section.find_elements(By.CLASS_NAME, "dFlex")

                if len(parts) >= 3:
                    offer_dict['salary'] = parts[0].text.strip()
                    offer_dict['duration'] = parts[1].text.strip()
                    offer_dict['schedule'] = parts[2].text.strip()
                else:
                    offer_dict['salary'] = offer_dict['duration'] = offer_dict['schedule'] = ""
            except:
                offer_dict['salary'] = offer_dict['duration'] = offer_dict['schedule'] = offer_dict['location'] = ""

            try:
                description_elem = driver.find_element(By.CLASS_NAME, "t_word_wrap")
                offer_dict['description'] = description_elem.text.strip()
            except:
                offer_dict['description'] = ""

            offer_dict['url'] = driver.current_url

            offers.append(offer_dict)

    except Exception as e:
        print(f"Erreur lors de la récupération des détails :\n{e}")

    return offers





In [ ]:
Dataclass = dataclasses.dataclass

@Dataclass
class Offer:
    title: str
    company: Optional[str] = None
    verified : bool = False
    location : str
    contract_type: str
    schedule: str
    location: str
    salary: Optional[int] = None